In [ ]:
# DL Assignment 3

# version 19.
# project name changed to final project name.
# added if condition for early stopping of less accuracy runs.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split


# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# device = "cpu"
print(device)

In [ ]:
!pip install wandb
import wandb
wandb.login(key = "c28eda24dacbd50a57332e445cfdfb2f4c16799c")


In [ ]:

# import wandb
wandb.init(project="DL_Assignment_3")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')


In [ ]:

path = '/kaggle/input/marathi/mar_train.csv'
path_val = '/kaggle/input/marathi/mar_valid.csv'
path_test = '/kaggle/input/marathi/mar_test.csv'
# path = '/content/drive/MyDrive/Colab Notebooks/DL Assignment/Assignment 3/aksharantar_sampled/aksharantar_sampled/mar/mar_train.csv'
# path_val = '/content/drive/MyDrive/Colab Notebooks/DL Assignment/Assignment 3/aksharantar_sampled/aksharantar_sampled/mar/mar_valid.csv'
# path_test = '/content/drive/MyDrive/Colab Notebooks/DL Assignment/Assignment 3/aksharantar_sampled/aksharantar_sampled/mar/mar_test.csv'

In [ ]:
df = pd.read_csv(path , header = None)
# df.head()
df_val = pd.read_csv(path_val , header = None)
df_test = pd.read_csv(path_test , header = None)
english_words_val = df_val[0]
marathi_words_val = df_val[1]
english_words_test = df_test[0]
marathi_words_test = df_test[1]
english_words = df[0]
marathi_words = df[1]
# print(english_words)
# print(marathi_words)
print(df[50000:])

In [ ]:
# creating list of charecters in both languages

english_char_list = []
max_length_word_english = -1
for word in english_words:
  max_length_word_english = max(max_length_word_english,len(word)) 
  for char in word :
    english_char_list.append(char);
english_char_list = list(set(english_char_list))
english_char_list.sort()
# print(len(english_char_list))
print(english_char_list)
# print(max_length_word_english)

marathi_char_list = []
max_length_word_marathi = -1
for word in marathi_words:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
  for char in word :
    marathi_char_list.append(char);
marathi_char_list = list(set(marathi_char_list))
marathi_char_list.sort()
# print(len(marathi_char_list))
print(marathi_char_list)


# finding out the maximum size word for english and marathi from validation and test data.
for word in english_words_val:
  max_length_word_english = max(max_length_word_english,len(word))
for word in english_words_test:
  max_length_word_english = max(max_length_word_english,len(word)) 
for word in marathi_words_val:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
for word in marathi_words_test:
  max_length_word_marathi = max(max_length_word_marathi,len(word))

print(max_length_word_english)
print(max_length_word_marathi)

In [ ]:
# english word to vector size = 27 ie. max_length_word_english
# marathi word to vector size = 20 ie. max_length_word_marathi
# for one word.
def word2vec(word, lang):
  vec = []
  if(lang == "english"):
    vec.append(len(english_char_list) + 1)
    for char in word:
      for i in range(len(english_char_list)):
        if(english_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_english + 1): # padding with max_length + 1.
#         vec.append(len(english_char_list) + 2)
        vec.append(0)
#     vec.append(len(english_char_list) + 2)
    vec.append(0)
  else :
    vec.append(len(marathi_char_list) + 1)
    for char in word:
      for i in range(len(marathi_char_list)):
        if( marathi_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_marathi + 1):  # padding with max_length + 1.
#         vec.append(len(marathi_char_list) + 2)
        vec.append(0)
#     vec.append(len(marathi_char_list) + 2)
    vec.append(0)
  return(vec)


In [ ]:
vec = word2vec(marathi_words[10],"marathi")
print(marathi_words[10])
print(vec)

In [ ]:
# creating matrix of representation for whole words of english and marathi.

def ip_matrix_construct(words, lang):
  ans = []
  for word in words:
    ans.append(word2vec(word, lang))
  return(ans)

In [ ]:
# calculated representations of whole english and marathi words in variables english and marathi matrix.
english_matrix = ip_matrix_construct(english_words, "english")
marathi_matrix = ip_matrix_construct(marathi_words, "marathi")
# english_matrix = english_matrix[:1000]
# marathi_matrix = marathi_matrix[:1000]
print(len(english_matrix))
print(len(marathi_matrix))
english_matrix = torch.tensor(english_matrix)
marathi_matrix = torch.tensor(marathi_matrix)
print(len(english_matrix[0]))
print(len(marathi_matrix[0]))

english_matrix_val = ip_matrix_construct(english_words_val, "english")
marathi_matrix_val = ip_matrix_construct(marathi_words_val, "marathi")
english_matrix_val = torch.tensor(english_matrix_val)
marathi_matrix_val = torch.tensor(marathi_matrix_val)
english_matrix_test = ip_matrix_construct(english_words_test, "english")
marathi_matrix_test =ip_matrix_construct(marathi_words_test, "marathi")
english_matrix_test = torch.tensor(english_matrix_test)
marathi_matrix_test = torch.tensor(marathi_matrix_test)

In [ ]:
# alaadin
# input size = number of characters in lang eg, english = 26 + 1 for start of sentence + 1 for end of sentence.
# embedding size  = hyperparamter
# hidden size = hyperparametrv 
# num layers  = hyperparametr
class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, enc_layers, p, cell_type, bidirectional):
    super(Encoder,self).__init__()
    self.hidden_size = hidden_size
    self.enc_layers = enc_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)

  def forward(self, x):
    # x shape: (seq_length, N)  where N is batch size
#     print("shape of x")
#     print(x.shape)
    embedding = self.dropout(self.embedding(x))
#     print("shape of embedding")
#     print(embedding.shape)
    # embedding shape : (seq_length, N, embedding_size)
    if(self.cell_type == "GRU"):
      output, hidden = self.gru(embedding)
    if(self.cell_type == "RNN"):
      output, hidden = self.rnn(embedding)
    if(self.cell_type == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      # print("hidden.shape",hidden.shape)
      # print("cell.shape",cell.shape)
      return outputs, hidden, cell
    # print("output.shape",output.shape)
    # print("hidden.shape",hidden.shape)
#     print(output.size())
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# alaadin
# input size = number of characters for target lang eg, marathi = 63
# embedidng size = will be sae as above.
# hidden size = same as above
# output size = same as input size ie. number of characters for target lang eg, marathi = 63
# num layers = same as above
class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
  
  def forward(self,x,output, hidden, cell = 0):
    # shape of x: (N) but we want (1,N)
    x = x.unsqueeze(0).int()
#     print("shape of x")
#     print(x.shape)
    embedding = self.dropout(self.embedding(x))
    # embedding shape : (1,N,embedding_size)
#     print("embedding shape")
#     print(embedding.shape)
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(embedding, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(embedding, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
    # shape of outputs: (1, N, hidden_size)
    predictions = self.fc(outputs)
    # shape of predictions: (1, N, length_of_vocab)
    predictions = predictions.squeeze(0)
    # shape of predictions: (N, length_of_vocab)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden


  def initHidden(self):
#     print("init hidden decoder")
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class Atten_decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type, bidirectional):
    super(Atten_decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    # self.dropout_p = dropout_p
    self.max_length = len(english_matrix[0])  #30
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(hidden_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
    self.attn = nn.Linear(hidden_size+embedding_size, self.max_length)
    if(bidirectional):
      self.attn_combine = nn.Linear(hidden_size * 2 + embedding_size, hidden_size)
    else :
      self.attn_combine = nn.Linear(hidden_size + embedding_size, hidden_size)

  def forward(self, x,output, hidden, cell = 0):
    x = x.unsqueeze(0)
    output=output.permute(1,0,2)
    embedded = self.embedding(x)
    embedded = self.dropout(embedded)
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0],hidden[0]), 1)), dim = 1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(1),output)
    attn_applied = attn_applied.squeeze(1)
    op = torch.cat((embedded[0], attn_applied), 1)

    op = self.attn_combine(op).unsqueeze(0)
    op = F.relu(op)
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(op, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(op, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(op, (hidden, cell))
    predictions = self.fc(outputs)
    # shape of predictions: (1, N, length_of_vocab)
    predictions = predictions.squeeze(0)
    # shape of predictions: (N, length_of_vocab)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden



In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, enc_layers, dec_layers):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
#         print("seq2seq intilized")

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(marathi_char_list) + 2  # intitially = 66 || Now 63 + 1(start token) + 1 = 65
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        if(self.cell_type == "LSTM"):
            encoder_output, hidden, cell = self.encoder(source)
        else:
            encoder_output, hidden = self.encoder(source)
        # if(self.bidirectional == True):
        if(self.enc_layers != self.dec_layers or self.bidirectional == True):
          hidden = hidden[self.enc_layers - 1] + hidden[self.enc_layers - 1]
          hidden = hidden.repeat(self.dec_layers,1,1)
          if(self.cell_type == "LSTM"):
              cell = cell[self.enc_layers - 1] + cell[self.enc_layers - 1]
              cell = cell.repeat(self.dec_layers,1,1)
        
        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]
    
        for t in range(1, target_len):
#             print("STARTED t= ",t)
            if(self.cell_type == "LSTM"):
                output, hidden, cell = self.decoder(x, encoder_output, hidden, cell)
            else :
                output, hidden = self.decoder(x, encoder_output, hidden)
#             print("output shape = ",output.shape)
#             print("hidden.shape = ", hidden.shape)
            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess
#             print("x.shape = ",x.shape)
#             print("COMPLETED t ", t)
            
#         print("decoder sucessful")
        return outputs


In [ ]:
def Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size):
    correct_count = 0
    for batch_idx in range((int)(len(english_matrix) / batch_size)):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        
        output = output.T
#         print(output[1])
#         print(output[1][1:])
#         print("output.shape = ",output.shape)
#         print("target.shape = ",target.shape)
        for i in range(batch_size):
            if(torch.equal(output[i][1:],target[i][1:])):
                correct_count += 1
    accuracy = correct_count * 100 / len(english_matrix)
    return accuracy
        

In [ ]:
def neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention):
  learning_rate = 1e-3
  num_epochs = 20
  input_size_encoder = len(english_char_list) + 2  # intially= 30 || Now 26 + 1(start token) + 1 = 28 
  input_size_decoder = len(marathi_char_list) + 2  # intitially = 66 || Now 63 + 1(start token) + 1 = 65
  output_size        = len(marathi_char_list) + 2  # intitially = 66 || Now 63 + 1(start token) + 1 = 65

  encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, enc_layers, enc_dropout, cell_type,bidirectional).to(device)
  if(attention):
    decoder_net = Atten_decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type, bidirectional).to(device)
  else:
    decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type).to(device)

  model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, enc_layers, dec_layers).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  pad_idx = len(marathi_char_list) + 1  # 64 # pading index for marathi
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


  for epoch in range(num_epochs):
      print("epoch = ",epoch)

      model.train()
      total_loss = 0
      step = 0
      for batch_idx in range((int)(len(english_matrix) / batch_size)):
          # Get input and targets and get to cuda
  #         inp_data = batch.src.to(device)
  #         target = batch.trg.to(device)

          # Forward prop
          inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
          target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
  #         inp_data = inp_data.T
          target = target.T
          output = model(inp_data.T, target)
  #         print("model successful")
  #         print("output.shape = ",output.shape)

          output = output[1:].reshape(-1, output.shape[2])
          target = target[1:].reshape(-1)
  #         print("output.shape = ",output.shape)
  #         print("target.shape = ",target.shape)
          optimizer.zero_grad()
          loss = criterion(output, target)
          total_loss += loss
          # Back prop
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

          # Gradient descent step
          optimizer.step()

          step += 1
  #     print("total loss = ",total_loss / (len(english_matrix) * len(marathi_matrix[0])))
      print("total loss = ",total_loss / step)
      training_accuracy = Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size)
      print("Training Accuracy = ", training_accuracy)
      val_accuracy = Accuracy(model, english_matrix_val, marathi_matrix_val, epoch, batch_size)
      print("Validation accuracy = ",val_accuracy)
      test_accuracy = Accuracy(model, english_matrix_test, marathi_matrix_test, epoch, batch_size)
      print("Test accuracy = ", test_accuracy)
      wandb.log({"train_accuracy": training_accuracy, "validation_accuracy": val_accuracy, "training_loss": total_loss / step,  'epoch': epoch})
      if(epoch > 10 and training_accuracy < 10):
        return
      if(epoch > 5 and training_accuracy < 1):
        return

In [ ]:
# cell_type = "RNN"
# bidirectional = True
# enc_layers = 3
# dec_layers = 2
# batch_size = 256
# embedding_size = 384
# hidden_size = 256
# dropout = 0.3
# enc_dropout = 0.3
# dec_dropout = 0.6
# attention = True


# neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention)

In [ ]:
def run_NN():
    

    # Initialize a new wandb run
    wandb.init()
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Local variables, values obtained from wandb config
    
    cell_type = config.cell_type
    attention = config.attention
    bidirectional = config.bidirectional
    enc_layers = config.enc_layers
    dec_layers = config.dec_layers
    batch_size = config.batch_size
    embedding_size = config.embedding_size
    hidden_size = config.hidden_size
    enc_dropout = config.enc_dropout
    dec_dropout = config.dec_dropout

    # Display the hyperparameters
    run_name = "ct_{}_at_{}_bd_{}_enl_{}_dnl_{}_bs_{}_es_{}_hs_{}_edo_{}_ddo_{}".format(cell_type, attention, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout)
    print(run_name)
    neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention)
    wandb.run.name = run_name
    wandb.run.save()

In [ ]:
sweep_config = {
  "name": "CS6910 Assignment 3",
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        
       
        "cell_type": {
            "values": ["GRU", "RNN", "LSTM"]
        },
        "attention": {
            "values": [True, False]
        },
        "bidirectional": {
            "values": [True, False]
        },
        "enc_layers": {
            "values": [1, 2, 3, 5]
        },
        "dec_layers": {
            "values": [1, 2, 3, 5]
        },
        "batch_size": {
            "values": [64, 128, 256]
        },
        "embedding_size": {
            "values": [16, 32, 64,128, 256, 384, 512]
        },
        "hidden_size": {
            "values": [16,32,64,128, 256, 384, 512]
        },
        "enc_dropout": {
            "values": [0,0.2,0.3,0.4,0.6]
        },
        "dec_dropout": {
            "values": [0,0.2,0.3,0.4,0.6]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, entity="rajmahajan24", project="DL_Assignment_3")
wandb.agent(sweep_id, run_NN, count=1200)